<H1>Dénombrement bactérien instantanné en Webcam _ Fialaire David : </H1></br>
<H2>Version 23 05 2020</H2>
<img src="Affiche.jpg"  width=350px alt="" />


<H2>* Recommandations d installation d'Opencv (indispensable pour le script) : </H2>
POUR INSTALLER OPENCV, il faut tenir compte de la version installée de Python..
Pour ma part, je suis Python 3.7.4 il faut donc que je récupère le fichier
Opencv contrib correspondant : "  opencv_python-4.1.2-cp37-cp37m-win_amd64.whl  "
.. en effet dans ce fichier on voit écrit   cp37 qui correspond à ma version python
.. le win pour windows et amd64 pour le 64bits
==> on trouve ces contrib au site suivant : "  https://www.lfd.uci.edu/~gohlke/pythonlibs/#opencv   "
il faut placer ce fichier téléchargé là ou se trouve votre fichier python.exe, pour
moi, cela se trouve ici : C:\ProgramData\Anaconda3
 --> Puis, ouvrir cmd en tant qu'administrateur ..
--> Placez-vous à l'endroit où vous avez copiez le ficher téléchargé précédemment avec la commande : cd C:\ProgramData\Anaconda3
Préparez le terrain pour Opencv en installant, (si ce n'est pas déjà fait !), numpy et matplotlib (pip install numpy, pip install matplotlib)
Enfin : faire : pip3 install opencv_python-4.1.2-cp37-cp37m-win_amd64.whl
ou : pip install opencv_python-4.1.2-cp37-cp37m-win_amd64.whl
Et voilà ! Redémarrer
Relancez un prompt, demandez python : >>>python
et écrire la commande python : import cv2   
Si tout se passe bien, il ne doit pas y avoir de message d'erreur !!

<H1>Mode d'emploi :</H1></br>
<H6>1) Commencez par activer la cellule Jupyter suivante (se placer à gauche de la cellule, cliquer gauche pour avoir la barre bleue, et faire Shift/Entrée)</H6>
Assurez vous, lorsque vous tapez une commande au clavier que ce soit la fenêtre Webcam qui soit sélectionnée</br> 
(si c'est une autre fenêtre, la commande ne sera pas prise en compte !)
<H6>2) Placer une boîte de Pétri à 10-15 cm de la webcam, sur un fond éclairé de manière homogène pour éviter le repérage d'éléments colorés du fond qui ne nous interresse pas.</H6>
<H3>Mode Calibrage : </H3>
<H6>3) Faîtes glisser la boîte de Pétri pour placer une colonie dans la cible rouge : le décompte se réalise automatiquement.</H6>
<H6>--> Vous pouvez aussi déplacer la cible rouge : (4: gauche) (6: droite) (8: haut) (2: bas)</H6>
<H3>Mode inverse : </H3>
<H6>4) Appuyez sur "n" pour passer en mode inverse, et sur "b" pour revenir en mode normal.</H6>
<H6>5) Placez alors la cible sur une zone homogène où il n'y a pas de colonies bactériennes.</H6>
<H3>Eclaircir et Assombrir : </H3>
<H6>6) Commandes "c" et "f"</H6>
<H3>Zoom et DéZoom : </H3>
<H6>6) Commandes "i" et "j"</H6>
<H3>Sortie : </H3>
<H6>7) Commandes "q"</H6>

In [4]:
import numpy as np
import cv2
from PIL import Image
from pylab import *
import matplotlib
from matplotlib.pyplot import *
import numpy
font = cv2.FONT_HERSHEY_SIMPLEX
inverse=0
###########################################
def fonction_p(img, lower_test, upper_test,inverse):
    modif=adjust_brightness(img, gamma)
    img=modif
    hauteur, largeur = img.shape[:2]
    # sortie d'image avec contours
    image_contours = img.copy()
    # Détection des colonies 
    colors = ['colonies']
    for color in colors:
        # copie de l'image originale
        image_to_process = img.copy()
        # initialisation du compteur
        counter[color] = 0
        # Dépôt des marges (GBR) de détection des couleurs dans un NumPy arrays
        if color == 'colonies':
            lower = np.array(lower_test)
            upper = np.array(upper_test)
        if inverse==1:
            image_to_process = (255-image_to_process)
        else:
            ok=2
        # Recherche des couleurs spécifiées dans les marges dans un masque
        image_mask2 = cv2.cvtColor(image_to_process, cv2.COLOR_BGR2HSV)
        image_mask = cv2.inRange(image_to_process, lower, upper)
        # application du masque
        image_res = cv2.bitwise_or(image_to_process, image_to_process, mask = image_mask)
        ## Transformation de l'image en nuance de gris et flouttage Gaussien
        image_gray = cv2.cvtColor(image_res, cv2.COLOR_BGR2GRAY)
        image_gray = cv2.GaussianBlur(image_gray, (5, 5), 0)
        # Détection des contours, et résolution des imperfections
        image_edged = cv2.Canny(image_gray, 50, 100)
        image_edged = cv2.dilate(image_edged, None, iterations=1)
        image_edged = cv2.erode(image_edged, None, iterations=1)
        cnts = cv2.findContours(image_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        # Boucles pour chaque contour détecté
        for c in cnts:
            # Si le contour n'est pas suffisamment large, l'ignorer : C'est un paramètre à modifier pour les tailles de colonies
            if cv2.contourArea(c) < 5:   ##1  #5
                continue
            # Création de la coqie convexe du contour
            hull = cv2.convexHull(c)
            if color == 'colonies':
                # Marquer le contour en vert
                cv2.drawContours(image_contours,[hull],0,(0,255,0),1) #0,0,255
            counter[color] += 1
            # Décocher le commentaire ci-dessous pour écrire, sur l'image, les TAILLES de colonies détectées
            #cv2.putText(image_contours, "{:.0f}".format(cv2.contourArea(c)), (int(hull[0][0][0]), int(hull[0][0][1])), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 0), 2)

    # Décocher le commentaire ci-dessous pour l'enregistrement de l'image finalisée
    #cv2.imwrite("Capture_denombrement.jpg",image_contours)     ######
    image_contours = image_contours.astype (np.uint8)
    return image_contours , counter[color]
#####################################################    
def fonction_m(img,x,y, inverse): 
    p_unmin=255
    p_deuxmin=255
    p_troismin=255
    p_unmax=0
    p_deuxmax=0
    p_troismax=0
    if inverse==1:
        img = (255-img)
    else:
        ok=2
    image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)      
    xpmin=x
    xpmax=x+6
    ypmin=y
    ypmax=y+6
    cible_img=image[xpmin:xpmax,ypmin:ypmax]
    for xp in range(xpmin,xpmax-1):
        for yp in range(ypmin,ypmax-1):
            p=img[xp,yp]
            Prep_pixel = np.uint8([[p]])
            hsv_pixel = cv2.cvtColor(Prep_pixel,cv2.COLOR_BGR2HSV)
            pi=hsv_pixel[0]
            p_un=pi[0][0]
            p_deux=pi[0][1]
            p_trois=pi[0][2]
            if p_un<p_unmin:
                p_unmin=p_un
            else:
                z=1
            if p_un>p_unmax:
                p_unmax=p_un
            else:
                z=1
            if p_deux<p_deuxmin:
                p_deuxmin=p_deux
            else:
                z=1
            if p_deux>p_deuxmax:
                p_deuxmax=p_deux
            else:
                z=1 
            if p_trois<p_troismin:
                p_troismin=p_trois
            else:
                z=1
            if p_trois>p_troismax:
                p_troismax=p_trois
            else:
                z=1 
    #lower_test=[p_unmin,25,25]
    #lower_test=[p_unmin,p_deuxmin,p_troismin] ## cool
    lower_test=[p_unmin-32,p_deuxmin,p_troismin]  ###
    #upper_test=[p_unmax,255,255]
    #upper_test=[p_unmax,p_deuxmax,p_troismax]  ## cool
    upper_test=[p_unmax+32,p_deuxmax,p_troismax]  ###
    #print("lower=",lower_test)
    #print("upper=",upper_test)
    return lower_test, upper_test
####################################################
def zoom_photo2(img):
    new_img=img[280:360,210:270]
    img = cv2.resize(new_img,(640,480))
    return img
####################################################
def cible_petite(img,xc,yc):
    xcmax=xc+6
    ycmax=yc+6
    for x in range(xc,xcmax):
        img[yc,x]=[0,0,255]
    for x in range(xc,xcmax):
        img[ycmax,x]=[0,0,255]

    for y in range(yc,ycmax):
        img[y,xc]=[0,0,255]
    for y in range(yc,ycmax):
        img[y,xcmax]=[0,0,255]
    return img
####################################################
def cible(img,yc,xc):
    xcmax=xc+64
    ycmax=yc+64
    for x in range(xc,xcmax):
        img[yc,x]=[0,255,0]
    for x in range(xc,xcmax):
        img[ycmax,x]=[0,255,0]

    for y in range(yc,ycmax):
        img[y,xc]=[0,255,0]
    for y in range(yc,ycmax):
        img[y,xcmax]=[0,255,0]
    return img
##################################################
def affich_image_array_pil(Matrice_Name):
    image = numpy.array(Matrice_Name)
    gray()
    # montrer l'image
    imshow(image)
#############################################
def adjust_brightness(img, brightness_factor):
    """Adjust brightness of an Image.
    Args:
        img (numpy ndarray): numpy ndarray to be adjusted.
        brightness_factor (float):  How much to adjust the brightness. Can be
            any non negative number. 0 gives a black image, 1 gives the
            original image while 2 increases the brightness by a factor of 2.
    Returns:
        numpy ndarray: Brightness adjusted image.
    """
    table = np.array([ i*brightness_factor for i in range (0,256)]).clip(0,255).astype('uint8')
    if img.shape[2]==1:
        return cv2.LUT(img, table)[:,:,np.newaxis]
    else:
        return cv2.LUT(img, table) 
##################################################
def transform_taille(img):
    img.thumbnail((640,480))
    return img
####################################################
def zoom_photo2(img):
    new_img=img[300:340,225:255]
    img = cv2.resize(new_img,(480,640))
    return img
####################################################
def zoom_photo(img,zoom):
    largeur = img.shape[0]
    hauteur = img.shape[1]   
    new_largeur=int(largeur/zoom)
    new_hauteur=int(hauteur/zoom)
    new_img = numpy.empty((new_largeur, new_hauteur, 3), dtype = numpy.uint8)
    for l in range (new_largeur):
        for h in range (new_hauteur):
            new_img[l,h]=img[l,h]
    img = cv2.resize(new_img,(640,480))  
    return img
#############################################
def affich(img,titre):
    cv2.imwrite(titre,img) ## modif
    Affich_fin = Image.open(titre)
    affich_image_array_pil(Affich_fin)
    title(titre)
    show()
#########################################
  # dictionnaire de comptage des colonies
counter = {}
##################################
# ça commence ici ################
##################################
""" on teste sans vidéocapture
mais en donnant l'image"""
#cam=cv2.VideoCapture(0)
img = cv2.imread("./images/test4.jpg", cv2.IMREAD_COLOR)
###############################
gamma=1
zoom=1
x = 306
y = 176
x2=317
y2=237
p_unmin=255
p_deuxmin=255
p_troismin=255
p_unmax=0
p_deuxmax=0
p_troismax=0
zoom2=0
while(True):
    ##############################
    """ on annule la lecture sur la cam"""
    #ret,img=cam.read()
    img = cv2.imread("./images/test4.jpg", cv2.IMREAD_COLOR)
    ##############################
    modif=adjust_brightness(img, gamma)
    img=modif
    if zoom2==1:
        img=zoom_photo2(img)
        img=cible(img,x,y)
        cv2.imshow("En direct _ Video Webcam", img)
    else:
        img=cible_petite(img,y2,x2)
        lower_test,upper_test=fonction_m(img,x2, y2,inverse)
        img,comptage=fonction_p(img, lower_test, upper_test,inverse)
        Texte1="On compte : "
        Texte2=" colonies."
        Texte=str(Texte1)+str(comptage)+str(Texte2)
        Texte3="inverse="
        Texte4=str(Texte3)+str(inverse)
        cv2.putText(img, Texte4 ,(20,30), font, 1, (0,0,0), 3);
        cv2.putText(img, Texte ,(20,60), font, 1, (0,0,0), 3);
        cv2.imshow("En direct _ Video Webcam", img)
#####################################################    
    if(cv2.waitKey(1) ==ord("n")):
        inverse=1
#####################################################    
    if(cv2.waitKey(1) ==ord("b")):
        inverse=0
#####################################################    
    if(cv2.waitKey(1) ==ord("i")):
        zoom2=1
#####################################################    
    if(cv2.waitKey(1) ==ord("j")):
        zoom2=0

#####################################################    
    if(cv2.waitKey(1) ==ord("6")):
        x2=x2+4
#####################################################    
    if(cv2.waitKey(1) ==ord("4")):
        x2=x2-4
#####################################################    
    if(cv2.waitKey(1) ==ord("8")):
        y2=y2-4
#####################################################    
    if(cv2.waitKey(1) ==ord("2")):
        y2=y2+4
#####################################################    
    if(cv2.waitKey(1) ==ord("z")):
        zoom=zoom*2
#####################################################    
    if(cv2.waitKey(1) ==ord("a")):
        zoom=zoom*0.5

#####################################################    
    if(cv2.waitKey(1) ==ord("q")):
        break
#####################################################  
    elif(cv2.waitKey(1) ==ord("f")):
        newgamma=gamma-0.1
        print("newgamma=",newgamma)
        if newgamma<0:
            print("désolé, on ne peut pas faire plus foncé !")
        else:
            gamma=newgamma
#####################################################            
    elif(cv2.waitKey(1) ==ord("c")):
        newgamma=gamma+0.1
        print("newgamma=",newgamma)
        if newgamma>2:
            print("désolé, on ne peut pas faire plus clair !")
        else:
            gamma=newgamma
#####################################################     
## Nécessité de détruire la fenêtre Webcam !
#############################################
cam.release()
cv2.destroyAllWindows()

In [ ]:
##################################
## Si ça bug, activer cette cellule
## pour fermer la fenêtre Webcam
##################################
cam.release()
cv2.destroyAllWindows()